- 변수 이름을 loop 돌려서 할당하기

https://m.blog.naver.com/PostView.nhn?blogId=nomadgee&logNo=220857820094&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error

import tensorflow as tf

In [2]:
os.getcwd()

'D:\\Mywork\\DACON_CONTEST_2019\\SUHYUN'

In [3]:
os.chdir('D:\\mywork\\DACON_CONTEST_2019')

df = pd.read_csv('.\\SOOMIN\\data\\TEST_DHweightDaconArima_NAFilled.csv')
df['Time'] = pd.to_datetime(df['Time']) 
df = df.set_index('Time')

In [4]:
df

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.296670
2017-08-29 11:00:00,1.458903,0.398,0.293,0.358,0.350,0.232,0.253,0.260,0.168,0.084,...,0.163,0.086,0.288,0.342010,1.150483,0.196,0.343,1.109,0.225,0.296670
2017-08-29 12:00:00,1.462694,0.427,0.383,0.462,0.410,0.228,0.243,0.429,0.146,0.054,...,0.192,0.181,0.323,0.341950,1.151945,0.111,0.322,1.109,0.225,0.296670
2017-08-29 13:00:00,1.465513,0.727,0.351,0.608,0.491,0.234,0.243,0.458,0.184,0.059,...,0.121,0.141,0.339,0.341925,1.153084,0.110,0.203,1.109,0.225,0.296670
2017-08-29 14:00:00,1.467609,1.208,0.336,0.454,0.404,0.242,0.228,0.333,0.146,0.055,...,0.214,0.096,0.241,0.341920,1.154028,0.106,0.133,1.109,0.225,0.296671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-30 19:00:00,1.255000,0.175,0.430,0.429,0.169,0.739,0.573,0.341,0.127,0.286,...,0.312,0.381,0.479,0.858000,11.546000,0.428,0.134,3.519,0.027,0.286000
2018-06-30 20:00:00,1.596000,1.692,0.332,0.366,0.159,0.255,0.501,0.281,0.126,0.230,...,0.327,0.402,0.725,0.816000,11.591000,0.413,0.206,3.331,0.027,0.303000
2018-06-30 21:00:00,1.379000,0.883,0.435,0.497,0.166,0.233,0.634,0.401,0.439,0.346,...,0.253,0.401,0.426,0.659000,10.766000,0.466,0.324,3.416,0.026,0.253000


In [5]:
df1 = df*1000 # kWh -> Wh로 변환

ConfigProto 문제 해결


https://stackoverflow.com/questions/56127592/attributeerror-module-tensorflow-has-no-attribute-configproto


Session 문제 해결


https://stackoverflow.com/questions/55142951/tensorflow-2-0-attributeerror-module-tensorflow-has-no-attribute-session

In [ ]:
# convert nparray
for k in range(len(df1.columns)):
    globals()['y_pred_list{}'.format(k)] = []
    nparr = df1.iloc[:, k].to_frame()
    nparr.astype('float32')
    print(nparr)

    # normalization
    scaler = MinMaxScaler(feature_range=(0, 1))
    nptf = scaler.fit_transform(nparr)
    
    # split train, test
    train_size = int(len(nptf) * 0.7)
    test_size = len(nptf) - train_size
    train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
    print(len(train), len(test))

    look_back = 24  
    # look_back = timesteip = seq_length
    data_dim = 1
    # data_dim ( 들어오는 데이터 열 개수)
    # output_dim = 24 (hour: 나가게 하려는 값)
    def create_dataset(dataset, look_back):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i + look_back)]
            dataX.append(a)
            dataY.append(dataset[i + look_back])
        return np.array(dataX), np.array(dataY)

    # create dataset for learning
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (-1, look_back, data_dim))
    testX = np.reshape(testX, (-1, look_back, data_dim))


    # simple lstm network learning
    model = Sequential()
    model.add(LSTM(4, input_shape=(look_back, data_dim))) 
    #  앞의 숫자는 Unit 개수
    # input_shape = (data_size, time_steps, features)  그러나 모델을 돌릴 땐, data_size는 따로 넘기지 않음.
    # batch_input_sshape = (data_size, time_steps, features) 주의점 data_size % batch_size == 0
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)


    # make prediction
    testPredict = model.predict(testX)
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform(testY)
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Train Score: %.2f RMSE' % testScore)

    # predict last value (or tomorrow?)
    lastX = nptf[-24:]
    lastX = np.reshape(lastX, (-1, look_back, data_dim))
    lastY_pred = model.predict(lastX)
    lastY = scaler.inverse_transform(lastY_pred)
    print('Predict the Close value of final day: %d' % lastY)  # 
    lastY_pred

    # predict last value (or tomorrow?)
    
    number = 1
    lastY_pred = 0
    for i in range(look_back):
        lastX = nptf[-look_back: ]
        lastX = np.reshape(lastX, (-1, look_back, data_dim))
        lastY_pred = model.predict(lastX)
        lastY = (scaler.inverse_transform(lastY_pred))/1000
        nptf = np.concatenate((nptf, lastY_pred))

        print('Predict value of column number {}: {}'.format(k, lastY[0][0]))  # 데이터 입력 마지막 다음날 종가 예측
        globals()['y_pred_list{}'.format(k)].append(lastY[0][0])

                             X26
Time                            
2017-08-29 10:00:00  1453.803354
2017-08-29 11:00:00  1458.902776
2017-08-29 12:00:00  1462.694299
2017-08-29 13:00:00  1465.513373
2017-08-29 14:00:00  1467.609412
...                          ...
2018-06-30 19:00:00  1255.000000
2018-06-30 20:00:00  1596.000000
2018-06-30 21:00:00  1379.000000
2018-06-30 22:00:00  1000.000000
2018-06-30 23:00:00   976.000000

[7334 rows x 1 columns]
5133 2201
Train on 5108 samples
Epoch 1/100
5108/5108 - 23s - loss: 5.8546e-04
Epoch 2/100
5108/5108 - 17s - loss: 4.4354e-04
Epoch 3/100
5108/5108 - 16s - loss: 3.4346e-04
Epoch 4/100
5108/5108 - 17s - loss: 3.3298e-04
Epoch 5/100
5108/5108 - 16s - loss: 3.2208e-04
Epoch 6/100
5108/5108 - 16s - loss: 3.0761e-04
Epoch 7/100
5108/5108 - 16s - loss: 2.4369e-04
Epoch 8/100
5108/5108 - 16s - loss: 1.7165e-04
Epoch 9/100
5108/5108 - 16s - loss: 1.1245e-04
Epoch 10/100
5108/5108 - 15s - loss: 4.5099e-05
Epoch 11/100
5108/5108 - 15s - loss: 2.6008

Epoch 28/100
5108/5108 - 15s - loss: 0.0078
Epoch 29/100
5108/5108 - 15s - loss: 0.0078
Epoch 30/100
5108/5108 - 15s - loss: 0.0078
Epoch 31/100
5108/5108 - 15s - loss: 0.0078
Epoch 32/100
5108/5108 - 15s - loss: 0.0078
Epoch 33/100
5108/5108 - 15s - loss: 0.0078
Epoch 34/100
5108/5108 - 15s - loss: 0.0078
Epoch 35/100
5108/5108 - 15s - loss: 0.0078
Epoch 36/100
5108/5108 - 15s - loss: 0.0078
Epoch 37/100
5108/5108 - 15s - loss: 0.0078
Epoch 38/100
5108/5108 - 15s - loss: 0.0078
Epoch 39/100
5108/5108 - 15s - loss: 0.0078
Epoch 40/100
5108/5108 - 15s - loss: 0.0078
Epoch 41/100
5108/5108 - 15s - loss: 0.0078
Epoch 42/100
5108/5108 - 15s - loss: 0.0078
Epoch 43/100
5108/5108 - 15s - loss: 0.0078
Epoch 44/100
5108/5108 - 15s - loss: 0.0078
Epoch 45/100
5108/5108 - 15s - loss: 0.0078
Epoch 46/100
5108/5108 - 15s - loss: 0.0078
Epoch 47/100
5108/5108 - 15s - loss: 0.0078
Epoch 48/100
5108/5108 - 15s - loss: 0.0078
Epoch 49/100
5108/5108 - 15s - loss: 0.0078
Epoch 50/100
5108/5108 - 15s - l

Epoch 75/100
5108/5108 - 15s - loss: 0.0042
Epoch 76/100
5108/5108 - 15s - loss: 0.0042
Epoch 77/100
5108/5108 - 15s - loss: 0.0042
Epoch 78/100
5108/5108 - 15s - loss: 0.0042
Epoch 79/100
5108/5108 - 15s - loss: 0.0042
Epoch 80/100
5108/5108 - 15s - loss: 0.0042
Epoch 81/100
5108/5108 - 15s - loss: 0.0042
Epoch 82/100
5108/5108 - 15s - loss: 0.0042
Epoch 83/100
5108/5108 - 15s - loss: 0.0042
Epoch 84/100
5108/5108 - 15s - loss: 0.0042
Epoch 85/100
5108/5108 - 15s - loss: 0.0042
Epoch 86/100
5108/5108 - 15s - loss: 0.0042
Epoch 87/100
5108/5108 - 16s - loss: 0.0042
Epoch 88/100
5108/5108 - 15s - loss: 0.0042
Epoch 89/100
5108/5108 - 15s - loss: 0.0042
Epoch 90/100
5108/5108 - 15s - loss: 0.0042
Epoch 91/100
5108/5108 - 15s - loss: 0.0042
Epoch 92/100
5108/5108 - 15s - loss: 0.0042
Epoch 93/100
5108/5108 - 15s - loss: 0.0042
Epoch 94/100
5108/5108 - 15s - loss: 0.0042
Epoch 95/100
5108/5108 - 15s - loss: 0.0042
Epoch 96/100
5108/5108 - 15s - loss: 0.0041
Epoch 97/100
5108/5108 - 15s - l

Predict value of column number 3: 0.33463260531425476
Predict value of column number 3: 0.34594786167144775
Predict value of column number 3: 0.3733174502849579
Predict value of column number 3: 0.3836292326450348
Predict value of column number 3: 0.37996771931648254
                      X402
Time                      
2017-08-29 10:00:00  432.0
2017-08-29 11:00:00  350.0
2017-08-29 12:00:00  410.0
2017-08-29 13:00:00  491.0
2017-08-29 14:00:00  404.0
...                    ...
2018-06-30 19:00:00  169.0
2018-06-30 20:00:00  159.0
2018-06-30 21:00:00  166.0
2018-06-30 22:00:00  463.0
2018-06-30 23:00:00  646.0

[7334 rows x 1 columns]
5133 2201
Train on 5108 samples
Epoch 1/100
5108/5108 - 17s - loss: 0.0050
Epoch 2/100
5108/5108 - 15s - loss: 0.0046
Epoch 3/100
5108/5108 - 15s - loss: 0.0046
Epoch 4/100
5108/5108 - 15s - loss: 0.0045
Epoch 5/100
5108/5108 - 16s - loss: 0.0044
Epoch 6/100
5108/5108 - 15s - loss: 0.0044
Epoch 7/100
5108/5108 - 15s - loss: 0.0044
Epoch 8/100
5108/5108 -

In [ ]:
y_pred_list0

In [ ]:
y_pred_list1

In [ ]:
for i in range(3):
    globals()['list{}'.format(i)] = []
    for k in range(4):
        globals()['list{}'.format(i)].append(k)

In [ ]:
y_pred_list

In [ ]:
list0